In [1]:
import os, json
import glob
import pandas as pd
import numpy as np
import time
import swifter
import datetime

In [2]:
"""
This part of the code deals with taking the user defined Local Directory path as input and giving a list of all the files 
for bookings data and airlines data.

Here we will also take the starttime and endtime input from the user and useit later in filtering our dataset.
"""
# this finds our json files
path_to_json = str(input('Please provide the Booking.json files path in the format drive/.../.../..: ')) #data/bookings
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)

# Now I will load the airlines data from the opensource file
path_to_dat = str(input('Please provide the airports.dat files path in the format drive/.../.../..: ')) #data/airports
dat_files = glob.glob(path_to_dat +'/*.dat')
print(dat_files)

# I am asking the user to provide the starttime and endtime which will be utilized later to filter the data.
# Get Start Date input
start_input = input('Enter a startdate in YYYY-MM-DD format')
syear, smonth, sday = map(int, start_input.split('-'))
startdate = datetime.date(syear, smonth, sday)
print(startdate)
# Get end date input
end_input = input('Enter an enddate in YYYY-MM-DD format')
eyear, emonth, eday = map(int, end_input.split('-'))
enddate = datetime.date(eyear, emonth, eday)
print(enddate)

Please provide the Booking.json files path in the format drive/filepath: data/bookings
['booking.json']
Please provide the airports.dat files path in the format drive/filepath: data/airports
['data/airports\\airports.dat']
Enter a startdate in YYYY-MM-DD format2019-03-01
2019-03-01
Enter an enddate in YYYY-MM-DD format2019-04-01
2019-04-01


In [3]:
# here I am defining a function which will be taking care of the Json structure and also loading the Json data
def fixing_json(files):
    global path_to_json
    #we need both the json and an index number so use enumerate()
    for index, js in enumerate(files):
        with open(os.path.join(path_to_json, js)) as json_file:
            # here I am fixing the issue with extra line we are getting because of format error in json data
            #Read whole file as string, store it into a variable.
            response = json_file.read()
            #Remove all occurrences of newlines and spaces.
            response = response.replace('\n', '')
            #Add the comma , the intersection }{, so it will be ...},{....
            response = response.replace('}{', '},{')
            #Encapsulate it with the brackets [].
            response = "[" + response + "]"
            #Use json.loads() to parse the JSON string.
            json_data = json.loads(response)
    return json_data

In [4]:
# here we are coverting the Json into a intermediate dataframe and also dropping the non required columns
json_text = fixing_json(json_files)
df_json_int = pd.json_normalize(json_text)
df_json_int = df_json_int[['timestamp', 'event.DataElement.travelrecord.passengersList',
                       'event.DataElement.travelrecord.productsList']]
                               
#df_json_int.head()

In [5]:
"""
Here I am trying to flatten the nested part of the dataframe created by normalizing the JSON file to create a df
using explode and normalize. 
The function will take the normalized df as input and then give the flattened df as an output.
The method I am trying to adopt in this is to use df columns of list type with explode function and that of dictionary type 
using normalize function. This loop will end once we don't have data for anyof the 2 type i.e. once we have a flattened file.

"""
# The function takes the dataframe as input. In our case it's the normalized df we created in earlier.
def flatten_json(df):
    df = df.reset_index()
    #here we are creating a df and applying type function = list type on each element of the df.
    s = (df.applymap(type) == list).all()
    # we are getting the values in a list
    list_columns = s[s].index.tolist()
    #here we are creating a df and applying type function = dict type on each element of the df.
    s = (df.applymap(type) == dict).all()
    # we are getting the values in a list
    dict_columns = s[s].index.tolist()

    # Initalizing a loop to get explode columns in list lype list and dict type list and append them in new columns list 
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []
        
        # this loop will normalize the list of dict type.
        for col in dict_columns:
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            #here we are concatinating the input df with the new df we have after normalizing and dropping the input columns
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            #getting the values from the list into the newly added columns
            new_columns.extend(horiz_exploded.columns) # inplace

        # This loop will explode the list of list type.
        for col in list_columns:
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)
        
        # Again we will be recursively using the same method to redefine the list and dict type columns in order to restart the
        # loop from the top and then normalize and explode the dict and list type columns respectively.
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()
    return df

# here we are finally getting the completely flattened json into a dataframe
booking_df = flatten_json(df_json_int)

In [6]:
# Working on dataframe to rename and drop the non required columns
booking_df = booking_df[['timestamp', 'event.DataElement.travelrecord.passengersList.age', 
                   'event.DataElement.travelrecord.passengersList.uci',
                  'event.DataElement.travelrecord.passengersList.passengerType', 
                   'event.DataElement.travelrecord.productsList.bookingStatus',
                  'event.DataElement.travelrecord.productsList.flight.operatingAirline',
                  'event.DataElement.travelrecord.productsList.flight.originAirport',
                   'event.DataElement.travelrecord.productsList.flight.destinationAirport',
                  'event.DataElement.travelrecord.productsList.flight.departureDate',
                  'event.DataElement.travelrecord.productsList.flight.arrivalDate']]
# Renaming the columns
booking_df.rename(columns={'event.DataElement.travelrecord.passengersList.age': 'age', 
                        'event.DataElement.travelrecord.passengersList.uci': 'uci',
                       'event.DataElement.travelrecord.passengersList.passengerType': 'passengerType',
                        'event.DataElement.travelrecord.productsList.bookingStatus':'bookingStatus',
                        'event.DataElement.travelrecord.productsList.flight.operatingAirline' : 'operatingAirline',
                        'event.DataElement.travelrecord.productsList.flight.originAirport' : 'originAirport',
                        'event.DataElement.travelrecord.productsList.flight.destinationAirport': 'destinationAirport',
                        'event.DataElement.travelrecord.productsList.flight.departureDate': 'departureDate',
                        'event.DataElement.travelrecord.productsList.flight.arrivalDate' : 'arrivalDate'}, inplace=True)

In [7]:
# Function to Load all open source data files. The columns name are defined as per information provided in the file. 
# Here I am creating a function which will parse the opensource airport file and then will assign the columns on it 
# The method I have used is to create a df with first file and then concat all the other files on that df
def opensource_df(files):
    columns = ['Airport_ID', 'Name', 'City', 'Country','IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 
               'Timezone', 'DST', 'Tz_Database_timezone', 'Type', 'Source']
    main_df = pd.DataFrame(pd.read_csv(files[0], names = columns, sep = ',', lineterminator='\n'))
    for i in range(1,len(files)):
        data_dat = pd.read_csv(files[i], sep=',', lineterminator='\n')
        df = pd.DataFrame(data_dat)
        main_df = pd.concat([main_df, df], axis = 1)
    
    return main_df

In [8]:
# Write all the dat files data in dataframeby using thefunction created above
airport_df = opensource_df(dat_files)

In [9]:
'''
Now I will join the data from the two sources based on the IATA code and the originAirport IATA code and create one merged 
Dataframe.
'''
combined_df = booking_df.merge(airport_df, how = 'left', left_on = 'originAirport', right_on = 'IATA')
#combined_df.shape

In [10]:
'''
Since I am only interested in KL flights we are filtering that out. Also, I am only looking into the flights taking off from
Netherlands which will also be filtered and a new Dataframe is created from these filters.
I have to analyze data for passengers who have confirmed booking only. So I will filter them out.
Input DF = combined_df
output DF = booking_kl_nl
'''
booking_kl_nl = combined_df[(combined_df.operatingAirline == 'KL') & (combined_df.Country == 'Netherlands') &
                           (combined_df['bookingStatus'] == 'CONFIRMED')].copy()
#booking_kl_nl.shape()

In [11]:
'''
As has been asked in the assignment that 'Each passenger should only be counted once per flight leg.'. Thus, I am removing all 
the duplicates.
'''
booking_kl_nl.drop_duplicates(subset= ['uci', 'departureDate'], inplace= True)

In [12]:
'''
I am reseting the index as we have filtered the data in our previous step. 
'''
booking_kl_nl.reset_index(drop= True, inplace=True)
#booking_kl_nl.head()

In [13]:
'''
Since I am asked to work as per the local timezone which in our case is Europe/Amsterdam provided in the Tz_Database_timezone.
But first I need to convert the time based columns into datetime format.
'''
booking_kl_nl['timestamp'] = pd.to_datetime(booking_kl_nl['timestamp'])
booking_kl_nl['departureDate'] = pd.to_datetime(booking_kl_nl['departureDate'])
booking_kl_nl['arrivalDate'] = pd.to_datetime(booking_kl_nl['arrivalDate'])

In [14]:
'''
The Next Step is to start applying the values in Tz_Database_timezone to the time based columns which are - timestamp (the time
when record was created), departureDate (The time of departure of the flight), arrivaldate -(Time of arrival of the flight). 
Since arrival date is not of concern for our results we will not be applying the timezone changes on them.

The reason I chose to apply the values in Tz_Database_timezone to make changes in the values of the time based columns was to 
make it generic rather than having the fixed value of 'Europe/Amsterdam'. 

Although this would have worked in our case as we had only one value in the Tz_Database_timezone column, but still I 
chose to make it generic.

I am using the swifter function from swifter library in order to utilize  Dask and Vectorize simultenously i.e. to parallelize 
apply on all the available machine cores. The partition can be handled based on the cpu count and number of processors we have
on any local machine. I am assuming that we have dual core machines (My local machine is dual core)
'''
#partitions = os.cpu_count()*2
#booking_kl_nl['timestamp'] = booking_kl_nl.swifter.set_npartitions(partitions).apply(
#    lambda row: row['timestamp'].tz_convert(row['Tz_Database_timezone']), axis =1)
#booking_kl_nl['departureDate'] = booking_kl_nl.swifter.set_npartitions(partitions).apply(
#    lambda row: row['departureDate'].tz_convert(row['Tz_Database_timezone']), axis =1)

booking_kl_nl['timestamp'] = booking_kl_nl.swifter.apply(
    lambda row: row['timestamp'].tz_convert(row['Tz_Database_timezone']), axis =1)
booking_kl_nl['departureDate'] = booking_kl_nl.swifter.apply(
    lambda row: row['departureDate'].tz_convert(row['Tz_Database_timezone']), axis =1)


In [15]:
'''
Now I will extrat just the dates, month and weekday from the departureDate column and then subsequently filter the data based on
the startdate and enddate user input for our analysis.
Also we will merge the data for airports back again with just the columns needed for our analysis in both the datasets.

'''
# Getting date.
booking_kl_nl['departure_date'] = booking_kl_nl['departureDate'].dt.date
#getting weekday name
booking_kl_nl['weekday'] = pd.Series(booking_kl_nl.departureDate).dt.day_name()
#getting month name
booking_kl_nl['month'] = booking_kl_nl['departureDate'].dt.month_name(locale='English')
#Filter based on start and end date
booking_kl_nl = booking_kl_nl.loc[(booking_kl_nl['departure_date'] >= startdate) & (booking_kl_nl['departure_date'] <= enddate)]
# Mereging the airports_df data in order to get country.
booking_kl_nl = pd.merge(booking_kl_nl[['timestamp','uci', 'departureDate', 'departure_date', 'destinationAirport',
                                       'weekday', 'month']], airport_df[['Country', 'IATA']], how = 'left', 
                         left_on = 'destinationAirport', right_on = 'IATA')
# Renaming the country to destination_country
booking_kl_nl.rename(columns = {'Country' : 'destination_country'}, inplace = True)

In [16]:
'''
The assignment says to represent the final table with season data in it. I couldn't find any season data in the Json file
or the dat file. So I assumed that the season data is to be manually found based on data available for seasons in netherlands
on the internet. 

I have created a dictionary for the season from the source : https://seasonsyear.com/Netherlands and then applied the values
to the newly created season colum in the dataset.
'''
# Created a dictionary
season_dict = {'January': 'Winter',
               'February': 'Winter',
               'March': 'Spring', 
               'April': 'Spring',
               'May': 'Spring',
               'June': 'Summer',
               'July': 'Summer',
               'August': 'Summer',
               'September': 'Fall',
               'October': 'Fall',
               'November': 'Fall',
               'December': 'Winter'}
# Create season column
booking_kl_nl['season'] = booking_kl_nl['month'].apply(lambda x: season_dict[x])

In [17]:
'''
Here I am grouping the total count of passengers based on destination_country, weekday and season as was asked in assignment.
The final output shows data in descending order sort.
'''
output_df = pd.DataFrame(booking_kl_nl.groupby(['destination_country'
                                                 , 'weekday', 'season'])['uci'].count().sort_values(ascending= False))
#Renaming the column uci to passenger_count
output_df.rename(columns= {'uci': 'passenger_count'}, inplace= True)
#Sorting the values based on season and weekday
output_df.sort_values(by=['season', 'weekday'], ascending=False)
pd.set_option('display.max_rows', 700)
output_df
#filename = 'Booking_Analysis-'+ str(datetime.date.today()) +'.csv'
#output_df.to_csv(filename)

passenger_count
destination_country  weekday   season                 
United States        Monday    Spring              117
United Kingdom       Sunday    Spring               97
                     Monday    Spring               92
France               Monday    Spring               91
China                Monday    Spring               86
                     Sunday    Spring               74
Italy                Wednesday Spring               67
Germany              Monday    Spring               64
United States        Sunday    Spring               63
United Kingdom       Thursday  Spring               48
United States        Tuesday   Spring               47
United Kingdom       Tuesday   Spring               45
Italy                Monday    Spring               42
United Kingdom       Saturday  Spring               40
                     Friday    Spring               38
Canada               Monday    Spring               36
Indonesia            Monday    Spring               34
Portugal             Wednesday Spring               33
Denmark              Monday    Spring               32
Germany              Tuesday   Spring               29
United Kingdom       Wednesday Spring               29
Sweden               Monday    Spring               28
Denmark              Sunday    Spring               27
Germany              Sunday    Spring               26
United Arab Emirates Sunday    Spring               25
France               Tuesday   Spring               22
Italy                Sunday    Spring               21
France               Wednesday Spring               21
Norway               Friday    Spring               20
France               Sunday    Spring               20
Norway               Monday    Spring               19
Chile                Sunday    Spring               19
South Africa         Monday    Spring               19
Spain                Thursday  Spring               18
Germany              Friday    Spring               18
                     Thursday  Spring               17
Malaysia             Monday    Spring               17
China                Friday    Spring               17
Italy                Tuesday   Spring               16
South Korea          Sunday    Spring               16
Spain                Monday    Spring               16
Germany              Wednesday Spring               16
Thailand             Sunday    Spring               15
Denmark              Friday    Spring               15
Spain                Tuesday   Spring               15
Ireland              Monday    Spring               14
Sweden               Sunday    Spring               14
China                Saturday  Spring               14
Brazil               Monday    Spring               14
Poland               Monday    Spring               13
Switzerland          Friday    Spring               12
Spain                Friday    Spring               12
Austria              Monday    Spring               12
United States        Saturday  Spring               11
Germany              Saturday  Spring               11
France               Friday    Spring               11
Switzerland          Thursday  Spring               11
Canada               Thursday  Spring               11
Sweden               Wednesday Spring               11
France               Thursday  Spring               10
United States        Wednesday Spring               10
Spain                Sunday    Spring               10
United States        Friday    Spring               10
Switzerland          Monday    Spring               10
Netherlands Antilles Sunday    Spring                9
Spain                Wednesday Spring                9
Norway               Sunday    Spring                9
Switzerland          Tuesday   Spring                9
Denmark              Thursday  Spring                9
Mauritius            Saturday  Spring                9
United States        Thursday  Spring                9
China                Wed

In [18]:
#There is  one thing that I am confused about for which I have written the following code.
#1. Reset index will change these indices in columns and the data will be spread.
#output_df.reset_index(inplace=True)